In [ ]:
import numpy as np
import pandas as pd
import math


In [ ]:
import os
import sys

root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)

In [ ]:
%load_ext autoreload
%autoreload 2
from strategy.Positions import UniV3Position, BiCurrencyPosition
from strategy.Portfolio import Portfolio
from strategy.Data import PoolDataUniV3
from strategy.Backtest import Backtest
from strategy.History import PortfolioHistory
from strategy.Strategies import BiCurrencyPassive, BiCurrencyActive, UniV3Passive, UniV3Active
from strategy.Viewers import PotrfolioViewer, RebalanceViewer

from strategy import Pool, Token, Fee, Frequency

In [ ]:
def tick_to_price(tick, decimal_diff=10):
    price = np.power(1.0001, tick) / 10 ** decimal_diff
    return price

def price_to_tick(price, decimal_diff=10):
    tick = math.log(price, 1.0001) + decimal_diff * math.log(10, 1.0001)
    return int(round(tick))

In [ ]:
pool = Pool(Token.WBTC, Token.WETH, Fee.MIDDLE)

In [ ]:
data = PoolDataUniV3.from_folder(pool)
data.preprocess()

In [ ]:
data.plot()

## Bi-currency passive strategy

In [ ]:
bi_curr_strat = BiCurrencyPassive(pool)

In [ ]:
b = Backtest(bi_curr_strat)
portfolio_history, rebalance_history = b.backtest(data.swaps)

In [ ]:
stats_df_sbc = portfolio_history.portfolio_stats()

In [ ]:
rv = RebalanceViewer(rebalance_history)

In [ ]:
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
f1, f2, f3, f4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
f1

In [ ]:
f2

In [ ]:
f3

In [ ]:
f4

## Bi-currency active strat 

In [ ]:
bi_curr_strat = BiCurrencyActive(600, 12, 18, pool, 0.01,  0.0002, 0.0002)

In [ ]:
b = Backtest(bi_curr_strat)
portfolio_history, rebalance_history = b.backtest(data.swaps)

In [ ]:
stats_df_bicur_active = portfolio_history.portfolio_stats()

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
fig1, fig2, fig3, fig4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig3

In [ ]:
fig4

## Passive UniV3 strategy

In [ ]:
univ3_passive = UniV3Passive(12, 18, pool, 0.01)

In [ ]:
b = Backtest(univ3_passive)
portfolio_history, rebalance_history = b.backtest(data.swaps)

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
stats_df_univ3_passive = hist_univ3_passive.portfolio_stats()

In [ ]:
fig1, fig2, fig3, fig4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig3

In [ ]:
fig2

In [ ]:
fig4

## Active UniV3 strategy

In [ ]:
lower_0 = 11
upper_0 = 19

mint_tolerance = 10
grid_width = 60
width_num = 1
# burn_num = 12

# burn_tolerance = grid_width * grid_num - mint_tolerance
# burn_tolerance = grid_width * (grid_num + burn_num) - mint_tolerance
burn_tolerance = 590 

univ3_active = UniV3Active(mint_tolerance, burn_tolerance, grid_width, width_num, lower_0, upper_0, pool, 0.01,)

In [ ]:
b = Backtest(univ3_active)
portfolio_history, rebalance_history = b.backtest(data.swaps)

In [ ]:
stats_df_univ3_active = hist_univ3_active.portfolio_stats()

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
fig1, fig2, fig3, fig4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig3

In [ ]:
fig4

## Aggregate results

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Portfolio Values
fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x=stats_df_sbc.index,
#         y=stats_df_sbc['portfolio_value_to_y'],
#         name="Bi-currency Simple",
#     ))

fig.add_trace(
    go.Scatter(
        x=stats_df_bicur_active.index,
        y=stats_df_bicur_active['portfolio_value_to_y'],
        name='Bi-currency balanced 2100',
    ))

fig.add_trace(
    go.Scatter(
        x=stats_df_univ3_passive.index,
        y=stats_df_univ3_passive['portfolio_value_to_y'],
        name='UniV3 Passive',
    ))

fig.add_trace(
    go.Scatter(
        x=stats_df_univ3_active.index,
        y=stats_df_univ3_active['portfolio_value_to_y'],
        name='UniV3 Active',
    ))

fig.update_xaxes(title_text="Timeline")
fig.update_yaxes(title_text="Value to Y")
fig.update_layout(title='Portfolio Values to Y')

In [ ]:
# Portfolio Performances
fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x=stats_df_sbc.index,
#         y=stats_df_sbc['portfolio_performance_to_y_to_year'],
#         name="Bi-currency Simple",
#     ))

fig.add_trace(
    go.Scatter(
        x=stats_df_bicur_active.index,
        y=stats_df_bicur_active['portfolio_performance_to_y_to_year'],
        name='Bi-currency balanced 2100',
    ))

fig.add_trace(
    go.Scatter(
        x=stats_df_univ3_passive.index,
        y=stats_df_univ3_passive['portfolio_performance_to_y_to_year'],
        name='UniV3 Passive',
    ))

fig.add_trace(
    go.Scatter(
        x=stats_df_univ3_active.index,
        y=stats_df_univ3_active['portfolio_performance_to_y_to_year'],
        name='UniV3 Active',
    ))

fig.update_xaxes(title_text="Timeline")
fig.update_yaxes(title_text="Performance in Y")
fig.update_layout(title='Portfolio Performances in Y')